# Unetのセグメンテーションモデル

In [ ]:
pip install segmentation_models_pytorch

In [ ]:
pip install pydicom

In [ ]:
pip install timm

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pydicom
import numpy as np
import os
import glob
from tqdm import tqdm
import gc

import torchvision
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from fastai.vision.all import *
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
CV = 5 #画像分類の学習に用いるクロスバリデーションの数
SEED = 777 #シード値
fold = 1 #検証用データのfold値
PATCH_SIZE = 512 #全体の画像の最終的なサイズ
patch_size = 64 #セグメンテーションをして切り取った部分の画像のサイズ
TH = .5 #全体の画像から病気であると判定した根拠の部分にマスク値を付けたときの閾値
SEG_TRAIN = False#セグメンテーションモデルを動かしたいときはTrueにする
SEG = {
    'BS':16,#セグメンテーションモデルのバッチサイズ
    'LR':5e-4,#セグメンテーションモデルの学習率
    'EPOCHS':10#セグメンテーションモデルのエポック数
}
INF = {
    'BS':64,#病気を判定するモデルのバッチサイズ
    'LR':1e-5,#病気を判定するモデルの学習率
    'EPOCHS':10#病気を判定するモデルのエポック数
}

In [ ]:
!unzip /content/drive/MyDrive/rsna-2024-lumbar-spine-degenerative-classification.zip -d /content #RSNA2024のzipファイルを今のディレクトリに展開する

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #結果の再現性を得るために各種シード値を固定するする関数

In [ ]:
class myUNet(nn.Module):
    def __init__(self):
        super(myUNet, self).__init__()

        self.UNet = smp.Unet(
            encoder_name="resnet18",#セグメンテーションモデルのバックボーンはresnet18を使用
            classes=5,#5種類のマスクをつける
            in_channels=1 #白黒画像なのでチャンネル数は1
        ).to(device)

    def forward(self,X):
        x = self.UNet(X)

        min_values = x.view(-1,5,PATCH_SIZE*PATCH_SIZE).min(-1)[0].view(-1,5,1,1)
        max_values = x.view(-1,5,PATCH_SIZE*PATCH_SIZE).max(-1)[0].view(-1,5,1,1)
        x = (x - min_values)/(max_values - min_values)#全体の画像のテンソルをminmaxscalingで正規化。テンソルのブロードキャスティングに注意すること。

        return x

In [ ]:
idx_map = torch.stack([torch.arange(PATCH_SIZE)]*PATCH_SIZE).to(device)
idx_map = torch.stack([idx_map,idx_map.T]).view(1,1,2,PATCH_SIZE,PATCH_SIZE)#画像の各地点にx座標の値とy座標の値を格納したものを用意する
class myLoss(nn.Module):
    def __init__(
            self,
            alpha=.5
        ):
        super().__init__()
        self.alpha = alpha

    def clone(self):
        return myLoss(self.alpha)
    def forward(
            self,
            y,# 予測される病気と判定するマスク値（0～1）
            t # 病気だと判定した部分の点座標値
        ):
        mask_pred = y
        _,mask_true = t
#       理想的な分布を中心（病気だと判定した部分の座標）から1変数の正規分布を３次元空間上で回転させたような分布（共分散行列がσ^2Iの2次元正規分布ともいえる）であると仮定する。
        s2 = torch.as_tensor([PATCH_SIZE/8]*5)#正規分布の分散(σ^2)のパラメタを設定(５つのマスクをまとめて)
#       正規分布のパラメタを以下のように設定する
        A = -1/(2*s2).to(device)#正規分布の式全体の係数の設定
        K = 1/torch.sqrt(2*math.pi*s2).to(device)#正規分布のexpの中の係数の設定
        mask_pred = mask_pred*K.view(1,5,1,1)#mask_predの最大値がKになるように合わせる。これは理想的な分布の最大値がKのためそこに合わせる。
        mask = idx_map - mask_true.view(-1,5,2,1,1)#病気だと判定した座標からのx座標とy座標の距離をmaskに格納
        mask = torch.exp((A.view(-1,5,1,1,1)*mask*mask).sum(2))*K.view(-1,5,1,1)#x座標とy座標の距離を使って距離に応じた正規分布の値を作りmaskに格納。
#       ロスは理想的な分布とのcosine類似度にする。コサイン類似度は2次元のものをそのまま1次元として扱って求める。
        D = 1 - ((mask*mask_pred).sum())**2/((mask*mask).sum()*(mask_pred*mask_pred).sum())

        return D

In [ ]:
train = pd.read_csv('/content/train.csv')#trainを読み込む
train.tail()

In [ ]:
diagnosis = list(filter(lambda x: x.find('foraminal') > -1, train.columns))
train = train[train[diagnosis].isnull().values.sum(1)==0].reset_index(drop=True)
train.tail()

In [ ]:
train2=train[['study_id']+diagnosis]

In [ ]:
labels = {
    'Normal/Mild':0,
    'Moderate':1,
    'Severe':2
}

In [ ]:
df_meta_f = pd.read_csv('/content/train_series_descriptions.csv')
df_meta_f.tail()

In [ ]:
df_coor = pd.read_csv('/content/train_label_coordinates.csv')
df_coor.tail()

In [ ]:
LF=df_coor[df_coor['condition']=='Left Neural Foraminal Narrowing'][[
    'study_id',
    'series_id',
    'instance_number',
    'level',
    'x',
    'y'
]].sort_values([
    'study_id',
    'series_id',
    'level'
])[[
    'study_id',
    'series_id',
    'level',
    'instance_number',
    'x',
    'y'
]]
LF.tail()

In [ ]:
LF = LF[[
    'study_id',
    'series_id',
    'instance_number',
    'x',
    'y'
]]

In [ ]:
LF[[
    'x_L1L2',
    'y_L1L2',
    'x_L2L3',
    'y_L2L3',
    'x_L3L4',
    'y_L3L4',
    'x_L4L5',
    'y_L4L5',
    'x_L5S1',
    'y_L5S1',
]] = np.tile(LF[['x','y']].values.reshape(-1,1,5,2),(1,5,1,1)).reshape(-1,10)
LF = LF.drop(columns=['x','y']).drop_duplicates().reset_index(drop=True)
LF.tail()

In [ ]:
diagnosis = list(filter(lambda x: x.find('left_neural') > -1, train.columns))

In [ ]:
LF = LF.merge(train[['study_id']+diagnosis], left_on='study_id', right_on='study_id')
LF.tail()

In [ ]:
diagnosis = {x:x[5:] for x in diagnosis}
diagnosis

In [ ]:
LF = LF.rename(columns=diagnosis)
LF.tail()

In [ ]:
RF = df_coor[df_coor['condition']=='Right Neural Foraminal Narrowing'][[
    'study_id',
    'series_id',
    'instance_number',
    'level',
    'x',
    'y'
]].sort_values([
    'study_id',
    'series_id',
    'level'
])[[
    'study_id',
    'series_id',
    'instance_number',
    'level',
    'x',
    'y'
]].drop_duplicates()
RF.tail()

In [ ]:
centers = {}
for i in range(len(RF)):
    row = RF.iloc[i]
    centers[row['study_id']]={}
for i in range(len(RF)):
    row = RF.iloc[i]
    centers[row['study_id']][row['series_id']]={'L1/L2':[],'L2/L3':[],'L3/L4':[],'L4/L5':[],'L5/S1':[]}
for i in range(len(RF)):
    row = RF.iloc[i]
    centers[row['study_id']][row['series_id']][row['level']].append([row['x'],row['y']])

In [ ]:
coordinates = np.zeros((len(RF),10))
coordinates[:] = np.nan
for i in range(len(RF)):
    row = RF.iloc[i]
    for level in centers[row['study_id']][row['series_id']]:
        if len(centers[row['study_id']][row['series_id']][level]) > 0:
            center = np.array(centers[row['study_id']][row['series_id']][level]).mean(0)
            coordinates[
                i,
                {'L1/L2':0, 'L2/L3':2, 'L3/L4':4, 'L4/L5':6, 'L5/S1':8}[level]:{'L1/L2':0, 'L2/L3':2, 'L3/L4':4, 'L4/L5':6, 'L5/S1':8}[level]+2
            ] = center

In [ ]:
RF = RF[[
    'study_id',
    'series_id',
    'instance_number',
    'x',
    'y'
]]
RF[[
    'x_L1L2',
    'y_L1L2',
    'x_L2L3',
    'y_L2L3',
    'x_L3L4',
    'y_L3L4',
    'x_L4L5',
    'y_L4L5',
    'x_L5S1',
    'y_L5S1',
]] = coordinates
RF = RF.drop(columns=['x','y']).drop_duplicates().reset_index(drop=True)
RF.tail()

In [ ]:
RF = RF[RF[[
    'x_L1L2',
    'y_L1L2',
    'x_L2L3',
    'y_L2L3',
    'x_L3L4',
    'y_L3L4',
    'x_L4L5',
    'y_L4L5',
    'x_L5S1',
    'y_L5S1',
]].isnull().values.sum(1)==0].reset_index(drop=True)
RF.tail()

In [ ]:
diagnosis = list(filter(lambda x: x.find('right_neural_foraminal') > -1, train.columns))
RF = RF.merge(train[['study_id']+diagnosis], left_on='study_id', right_on='study_id')
RF.tail()

In [ ]:
diagnosis = {x:x[6:] for x in diagnosis}
diagnosis

In [ ]:
RF = RF.rename(columns=diagnosis)
RF.tail()

In [ ]:
F = pd.concat([LF,RF],axis=0,ignore_index=True)
F.tail()

In [ ]:
F = F.merge(df_meta_f[['series_id','series_description']], left_on='series_id', right_on='series_id')
F.tail()

In [ ]:
target = F.columns[-6:-1]
target

In [ ]:
labels = {
    'Normal/Mild':0,
    'Moderate':1,
    'Severe':2
}

coor = [
    'x_L1L2',
    'y_L1L2',
    'x_L2L3',
    'y_L2L3',
    'x_L3L4',
    'y_L3L4',
    'x_L4L5',
    'y_L4L5',
    'x_L5S1',
    'y_L5S1',
]

In [ ]:
class T1Dataset(Dataset):
    def __init__(self, df, VALID=False, alpha=0):
        self.data = df
        self.VALID = VALID
        self.alpha = alpha

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]#1行目を取得

        centers = torch.as_tensor([x for x in row[coor]]).view(5,2).float()#病気だと判定した中心座標を取得

        sample = '/content/train_images/'
        sample = sample+str(row['study_id'])+'/'+str(row['series_id'])+'/'+str(row['instance_number'])+'.dcm'
        image = pydicom.dcmread(sample).pixel_array#pydicomで画像を読み込む
        H,W = image.shape

#画像を正方形になるように切り取る。切り取った分中心座標の位置を調整する。
        if H > W:
            d = W
            if not self.VALID:
                h = int((H - d)*(.5 + self.alpha*(.5 - np.random.rand())))
            else:
                h = (H - d)//2
            image = image[h:h+d]
            centers[:,1] -= h
            H = W
        elif H < W:
            d = H
            if not self.VALID:
                w = int((W - d)*(.5 + self.alpha*(.5 - np.random.rand())))
            else:
                w = (W - d)//2
            image = image[:,w:w+d]
            centers[:,0] -= w
            W = H

        image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))#画像をPATCH_SIZE*PATCH_SIZEにする
        image = torch.as_tensor(image/np.max(image)).unsqueeze(0).float()

        label = torch.as_tensor([labels[x] for x in row[target]])#labelは使用しないが適当になにかしら入れておく。

        #中心座標の位置を調整する。

        centers[:,0] = centers[:,0]*PATCH_SIZE/W
        centers[:,1] = centers[:,1]*PATCH_SIZE/H

        return image.to(device),[label.to(device),centers.to(device)]

In [ ]:
SEG_TRAIN = True#セグメンテーションの学習をしたいときはここをTrueにする。

In [ ]:
import pickle

tdf = F[F['series_description'] != fold]#trainのデータを持ってくる。
vdf = F[F['series_description'] == fold]#validのデータを持ってくる。

# tdf2=df_melted[df_melted['series_description'] != fold]
# vdf2=df_melted[df_melted['series_description'] == fold]

tds = T1Dataset(tdf)#定義したdataset関数からtrainのデータを持ってくる。
vds = T1Dataset(vdf,VALID=True)#定義したdataset関数からvalidのデータを持ってくる。
tdl = torch.utils.data.DataLoader(tds, batch_size=SEG['BS'], shuffle=True, drop_last=True)#バッチサイズを指定してデータローダー化する
vdl = torch.utils.data.DataLoader(vds, batch_size=SEG['BS'], shuffle=False)#バッチサイズを指定してデータローダー化する

if SEG_TRAIN:
    seed_everything(SEED)

    dls = DataLoaders(tdl,vdl)#fastai用にデータローダーを定義する。

    n_iter = len(tds)//SEG['BS']

    model = myUNet()
    learn = Learner(
        dls,
        model,
        lr=SEG['LR'],#学習率を設定
        loss_func=myLoss(alpha=0.5),
        # cbs=[
        #     ShowGraphCallback(),#学習曲線を表示
        #     alpha_cb
        # ]
    )
    learn.fit_one_cycle(SEG['EPOCHS'])#エポック数を設定
    with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+str(fold)+"_sagt1_rev"+".pkl", 'wb') as f:#モデルを回したら保存する。
      pickle.dump(model, f)
    del tdl,vdl,dls,model,learn #終わったらモデルを全て消す（メモリを使いすぎないため）
    gc.collect()

# saggitalT1画像の前処理

In [ ]:
import os
import pandas as pd

# データセットのパス
base_dir = '/content/train_images'

# リストを初期化
data = []

# 一番上の階層（study_id）のフォルダをたどる
for study_id in os.listdir(base_dir):
    study_path = os.path.join(base_dir, study_id)
    if os.path.isdir(study_path):
        # 二番目の階層（series_id）のフォルダをたどる
        for series_id in os.listdir(study_path):
            series_path = os.path.join(study_path, series_id)
            if os.path.isdir(series_path):
                # 三番目の階層（〇〇.dcm ファイル）をたどる
                for filename in os.listdir(series_path):
                    if filename.endswith('.dcm'):
                        # 〇〇.dcm の〇〇部分（instance_number）を抽出
                        instance_number = filename.split('.')[0]
                        # データを追加
                        data.append([study_id, series_id, instance_number])

# pandas DataFrameを作成
df = pd.DataFrame(data, columns=['study_id', 'series_id', 'instance_number'])



In [ ]:
df_meta_f = pd.read_csv('/content/train_series_descriptions.csv')
df_meta_f.tail()

df = df.astype('int64')

all_df = pd.merge(df, df_meta_f, on=['study_id', 'series_id'], how='inner')

s_all_df=all_df[all_df['series_description']=='Sagittal T1']

In [ ]:
import os
import numpy as np

# データフレームをコピーして操作
s_all_df_copy = s_all_df.copy()

# x_pos カラムを NaN で初期化
s_all_df_copy['x_pos'] = None
s_all_df_copy['y_pos'] = None
s_all_df_copy['z_pos'] = None
s_all_df_copy['pixel_sp_z'] = None
s_all_df_copy['pixel_sp_y'] = None
# 各行を処理
for idx, row in s_all_df_copy.iterrows():
    # DICOMファイルのパスを構築
    dicom_file_path = f"/content/train_images/{row['study_id']}/{row['series_id']}/{row['instance_number']}.dcm"

    # DICOMファイルを読み込む
    dicom_data = pydicom.dcmread(dicom_file_path)

    # 'Image Position (Patient)' の x 座標を取得して x_pos カラムに格納
    s_all_df_copy.loc[idx, 'x_pos'] = dicom_data.ImagePositionPatient[0]
    s_all_df_copy.loc[idx, 'y_pos'] = dicom_data.ImagePositionPatient[1]
    s_all_df_copy.loc[idx, 'z_pos'] = dicom_data.ImagePositionPatient[2]
    s_all_df_copy.loc[idx, 'pixel_sp_z']=dicom_data.PixelSpacing[0]
    s_all_df_copy.loc[idx, 'pixel_sp_y']=dicom_data.PixelSpacing[1]



In [ ]:
import pandas as pd

# 骨の画像を正しい順番（手から体の中心部に行くような順番）に並び替える
def assign_description(group):
    # x_posを基準に小さい順に並び替え
    group = group.sort_values(by='x_pos').reset_index(drop=True)

    # 前半と後半を分割するインデックス
    midpoint = len(group) // 2

    # 前半を'Right Neural Foraminal Narrowing'
    group.loc[:midpoint-1, 'description'] = 'Right Neural Foraminal Narrowing'

    # 後半を'Left Neural Foraminal Narrowing'
    group.loc[midpoint:, 'description'] = 'Left Neural Foraminal Narrowing'

    return group

# study_id, series_idごとにグループ化し、assign_description関数を適用
s_all_df = s_all_df_copy.groupby(['study_id']).apply(assign_description).reset_index(drop=True)

In [ ]:
def sort_based_on_description(df):
    # "Right Neural Foraminal Narrowing" は x_pos が小さい順
    df_right = df[df['description'] == 'Right Neural Foraminal Narrowing'].sort_values(by='x_pos', ascending=True)

    # "Left Neural Foraminal Narrowing" は x_pos が大きい順
    df_left = df[df['description'] == 'Left Neural Foraminal Narrowing'].sort_values(by='x_pos', ascending=False)

    # 結合して返す
    return pd.concat([df_right, df_left])

# study_id, series_idごとにグループ化し、上記の関数で並び替え
sorted_s_all_df2 = s_all_df.groupby(['study_id', 'series_id'], group_keys=False).apply(sort_based_on_description)

# 結果を表示

In [ ]:
import pandas as pd

# グループごとに均等な間隔で10個の行を選択する関数
def select_evenly_spaced(group, num=8):
    # グループのサイズが10個未満の場合はそのまま返す
    if len(group) <= num:
        return group
    # ステップを計算し、均等な間隔でデータを選ぶ
    step = len(group) / num
    indices = [int(i * step) for i in range(num)]
    return group.iloc[indices]

# study_id, instance_number, descriptionでグループ化し、各グループから均等に10個選択
new_df =sorted_s_all_df2.groupby(['study_id', 'description'], group_keys=False).apply(select_evenly_spaced)

# 結果の確認
new_df.head(20)

In [ ]:
import pandas as pd
import numpy as np

# 各study_idグループに8個以上の行が入るようにするための処理
def ensure_min_rows(group, min_rows=8):
    if len(group) < min_rows:
        #8個より少ない場合はランダムにそのグループ内から8個になるように付け足す。
        additional_rows = group.sample(n=min_rows - len(group), replace=True, random_state=42)
        group = pd.concat([group, additional_rows], ignore_index=True)
    return group

new_df_padded = new_df.groupby(['study_id', 'description'], group_keys=False).apply(ensure_min_rows)

print(new_df_padded)

In [ ]:
import pandas as pd

# s_all_df3の5つのコピーを作成し、それぞれにlevelカラムを追加
dfs = [new_df_padded.copy().assign(level=i) for i in range(5)]

# それらのデータフレームを縦に連結
combined_df = pd.concat(dfs, ignore_index=True)

# 結果の確認
print(combined_df.head())
print(combined_df['level'].value_counts())

In [ ]:
merged_df = pd.merge(combined_df,train, on='study_id', how='left')

In [ ]:
import pandas as pd
import numpy as np

# サンプルデータフレームの作成（ここでは仮のカラムを使用しています）
# 実際には combined_df には適切なカラムが含まれている必要があります
# combined_df = pd.DataFrame({
#     'description': [...],
#     'level': [...],
#     'left_neural_foraminal_narrowing_l1_l2': [...],
#     'left_neural_foraminal_narrowing_l2_l3': [...],
#     'left_neural_foraminal_narrowing_l3_l4': [...],
#     'left_neural_foraminal_narrowing_l4_l5': [...],
#     'left_neural_foraminal_narrowing_l5_s1': [...],
#     'right_neural_foraminal_narrowing_l1_l2': [...],
#     'right_neural_foraminal_narrowing_l2_l3': [...],
#     'right_neural_foraminal_narrowing_l3_l4': [...],
#     'right_neural_foraminal_narrowing_l4_l5': [...],
#     'right_neural_foraminal_narrowing_l5_s1': [...],
# })

# labelにはtrain.csvで明示されている正解データを格納する
def get_label(row):
    if row['description'] == 'Left Neural Foraminal Narrowing':
        if row['level'] == 0:
            return row['left_neural_foraminal_narrowing_l1_l2']
        elif row['level'] == 1:
            return row['left_neural_foraminal_narrowing_l2_l3']
        elif row['level'] == 2:
            return row['left_neural_foraminal_narrowing_l3_l4']
        elif row['level'] == 3:
            return row['left_neural_foraminal_narrowing_l4_l5']
        elif row['level'] == 4:
            return row['left_neural_foraminal_narrowing_l5_s1']
    elif row['description'] == 'Right Neural Foraminal Narrowing':
        if row['level'] == 0:
            return row['right_neural_foraminal_narrowing_l1_l2']
        elif row['level'] == 1:
            return row['right_neural_foraminal_narrowing_l2_l3']
        elif row['level'] == 2:
            return row['right_neural_foraminal_narrowing_l3_l4']
        elif row['level'] == 3:
            return row['right_neural_foraminal_narrowing_l4_l5']
        elif row['level'] == 4:
            return row['right_neural_foraminal_narrowing_l5_s1']
    return np.nan  # 予期しない値の場合はNaNを返す

# 新しいカラム `label` を作成
merged_df['label'] = merged_df.apply(get_label, axis=1)

# 結果の確認
print(combined_df.head())

In [ ]:
combined_df=merged_df

In [ ]:
import pandas as pd

# 例として final_df を作成（実際には適切なデータフレームを使用します）
# final_df = pd.DataFrame({
#     'description': [...],
#     'level': [...],
#     'label': [...],
#     'instance_number': [...],
#     'other_col1': [...],
#     'other_col2': [...],
#     ...
# })

# 1行にまとめるための処理

def group_rows(df, group_size):
    grouped = []
    num_groups = len(df) // group_size + int(len(df) % group_size != 0)

    for i in range(num_groups):
        start_idx = i * group_size
        end_idx = min((i + 1) * group_size, len(df))
        subset = df.iloc[start_idx:end_idx].reset_index(drop=True)

        # グループ内の先頭行の値を取得
        first_row = subset.iloc[0]

        # 新しいカラムに instance_number を格納
        row_data = first_row.to_dict()  # 他の列の情報も含めて最初の行の情報を取得
        row_data.update({
            f'instance_number_{j}': subset.iloc[j]['instance_number'] for j in range(len(subset))
        })

        # フィルの中身をリストに格納
        grouped.append(row_data)

    return pd.DataFrame(grouped)

# グループサイズを設定（5つの行を1行にまとめる）
group_size = 8
grouped_df = group_rows(combined_df, group_size)

print(grouped_df.head())

In [ ]:
grouped_df=grouped_df[grouped_df['label'].notna()]#ラベルがついていないものは排除

In [ ]:
grouped_df.reset_index(drop=True, inplace=True)

In [ ]:
# DataFrameを保存するパス
save_path = '/content/drive/MyDrive/RSNA_csv/grouped_df_rev.pkl'

# DataFrameをpkl形式で保存
grouped_df.to_pickle(save_path)

print(f"DataFrame has been saved to {save_path}")

# saggitalT1画像の真ん中だけを取ってきたものの前処理

In [ ]:
import pandas as pd

# s_all_dfをstudy_idごとにグループ化し、instance_numberでソートし、真ん中の要素を取得
def get_middle_instance(df):
    middle_rows = []

    # study_idごとにグループ化
    grouped = df.groupby('study_id')

    for _, group in grouped:
        # instance_numberで昇順にソート
        sorted_group = group.sort_values(by='instance_number').reset_index(drop=True)

        # 真ん中のインデックスを計算
        midpoint = len(sorted_group) // 2

        # 真ん中の行を取得
        middle_row = sorted_group.iloc[midpoint]

        # 真ん中の行をリストに追加
        middle_rows.append(middle_row)

    # すべての真ん中の行を結合して新しいDataFrameを作成
    middle_df = pd.DataFrame(middle_rows)

    return middle_df

middle_s_all_df = get_middle_instance(s_all_df_copy)

# 結果を表示
print(middle_s_all_df)


In [ ]:
middle_s_all_df =  middle_s_all_df.reset_index(drop=False)

In [ ]:
with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+"1"+".pkl", 'rb') as f:
      model1=pickle.load(f)

In [ ]:
import torch

# (OUT > 0.5)[0][1] がテンソルマスクだと仮定
for j in range(len(middle_s_all_df)):
  image=pydicom.dcmread("/content/train_images/"+str(middle_s_all_df.iloc[j]["study_id"])+"/"+str(middle_s_all_df.iloc[j]["series_id"])+"/"+str(middle_s_all_df.iloc[j]["instance_number"])+".dcm").pixel_array
  dicom=pydicom.dcmread("/content/train_images/"+str(middle_s_all_df.iloc[j]["study_id"])+"/"+str(middle_s_all_df.iloc[j]["series_id"])+"/"+str(middle_s_all_df.iloc[j]["instance_number"])+".dcm")
  H,W = image.shape
  middle_s_all_df.loc[j, 'H'] = H
  middle_s_all_df.loc[j, 'W'] = W
  middle_s_all_df.loc[j,'o0']=dicom.ImageOrientationPatient[0]
  middle_s_all_df.loc[j,'o1']=dicom.ImageOrientationPatient[1]
  middle_s_all_df.loc[j,'o2']=dicom.ImageOrientationPatient[2]
  middle_s_all_df.loc[j,'o3']=dicom.ImageOrientationPatient[3]
  middle_s_all_df.loc[j,'o4']=dicom.ImageOrientationPatient[4]
  middle_s_all_df.loc[j,'o5']=dicom.ImageOrientationPatient[5]
  middle_s_all_df.loc[j, 'H_cut']=0
  middle_s_all_df.loc[j, 'W_cut']=0
  # centers = torch.as_tensor([x for x in row[coor]]).view(5,2).float()
  # By plane resizing I've been distorting the proportions
  if H > W:
      d = W
      h = (H - d)//2
      image = image[h:h+d]
      middle_s_all_df.loc[j, 'H_cut'] = h
      # centers[:,1] -= h
      H = W
  elif H < W:
      d = H
      w = (W - d)//2
      image = image[:,w:w+d]
      middle_s_all_df.loc[j, 'W_cut'] = w
      # centers[:,0] -= w
      W = H
  image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
  image = torch.as_tensor(image/np.max(image)).unsqueeze(0).float()
  image=image.to(device)
  OUT = model1(image.unsqueeze(0)).cpu().detach()
  for i in range(5):
    mask = (OUT > 0.5)[0][i]  # 512x512のテンソル

    # Trueのピクセル座標を取得（y座標とx座標のペアとして取得）
    true_pixels = torch.nonzero(mask)

    # 平均座標を計算
    mean_yx = true_pixels.float().mean(dim=0)

    # 平均座標を表示
    mean_y, mean_x = mean_yx[0], mean_yx[1]
    print(f"Mean coordinates: (y, x) = ({0-mean_y.item()}, {mean_x.item()})")

    middle_s_all_df.loc[j, f'mean_y_{i}']=mean_y.item()
    middle_s_all_df.loc[j, f'mean_x_{i}']=mean_x.item()

    middle_s_all_df.loc[j, f'z_pos_{i}'] = middle_s_all_df.loc[j, 'pixel_sp_z'] * (0 - mean_y.item()) + middle_s_all_df.loc[j, 'z_pos']


In [ ]:
import torch

# 各マスクの座標を元の画像サイズに戻すコード
for j in range(len(middle_s_all_df)):
    # 元の画像サイズ
    H = middle_s_all_df.loc[j, 'H']
    W = middle_s_all_df.loc[j, 'W']

    # 512にリサイズされた画像のサイズ
    resized_size = 512

    # 高さと幅のスケール（リサイズ前の座標に戻すためのスケール）
    if H<W:
      scale_y = H / resized_size
      scale_x = H / resized_size
    elif H>W:
      scale_y = W / resized_size
      scale_x = W / resized_size
    else:
      scale_y = W / resized_size
      scale_x = H / resized_size
    # 5つのマスクについて座標を戻す
    for i in range(5):
        # リサイズ後の座標を取得
        mean_y_resized = middle_s_all_df.loc[j, f'mean_y_{i}']
        mean_x_resized = middle_s_all_df.loc[j, f'mean_x_{i}']

        # リサイズ前の座標に戻す（スケールを掛ける）
        mean_bfr_y = mean_y_resized * scale_y
        mean_bfr_x = mean_x_resized * scale_x

        # 戻した座標を新しいカラムに格納
        middle_s_all_df.loc[j, f'mean_bfr_y_{i}'] = mean_bfr_y
        middle_s_all_df.loc[j, f'mean_bfr_x_{i}'] = mean_bfr_x


In [ ]:
import torch

# 各マスクの切り取る前の座標を計算して新しいカラムに格納
for j in range(len(middle_s_all_df)):
    # H_cut, W_cut はどれくらい切り取られたかを表す（NaNの場合は切り取りなし）
    H_cut = middle_s_all_df.loc[j, 'H_cut'] if not pd.isna(middle_s_all_df.loc[j, 'H_cut']) else 0
    W_cut = middle_s_all_df.loc[j, 'W_cut'] if not pd.isna(middle_s_all_df.loc[j, 'W_cut']) else 0

    for i in range(5):
        # リサイズ前の平均座標を取得
        mean_bfr_y = middle_s_all_df.loc[j, f'mean_bfr_y_{i}']
        mean_bfr_x = middle_s_all_df.loc[j, f'mean_bfr_x_{i}']

        # 切り取る前の座標を計算（H_cutとW_cutを戻す）
        mean_bfr_cut_y = mean_bfr_y + H_cut
        mean_bfr_cut_x = mean_bfr_x + W_cut

        # 切り取る前の座標を新しいカラムに格納
        middle_s_all_df.loc[j, f'mean_bfr_cut_y_{i}'] = mean_bfr_cut_y
        middle_s_all_df.loc[j, f'mean_bfr_cut_x_{i}'] = mean_bfr_cut_x


In [ ]:
import numpy as np

# 3次元座標を計算してカラムに格納するコード
for j in range(len(middle_s_all_df)):
    # DICOMのメタデータ
    x_pos = middle_s_all_df.loc[j, 'x_pos']
    y_pos = middle_s_all_df.loc[j, 'y_pos']
    z_pos = middle_s_all_df.loc[j, 'z_pos']

    o0, o1, o2 = middle_s_all_df.loc[j, ['o0', 'o1', 'o2']]  # 行方向ベクトル (ImageOrientationPatientの0〜2番目)
    o3, o4, o5 = middle_s_all_df.loc[j, ['o3', 'o4', 'o5']]  # 列方向ベクトル (ImageOrientationPatientの3〜5番目)

    pixel_sp_y = middle_s_all_df.loc[j, 'pixel_sp_y']  # 行方向のPixelSpacing
    pixel_sp_z = middle_s_all_df.loc[j, 'pixel_sp_z']  # 列方向のPixelSpacing

    # 行方向ベクトル
    row_direction = np.array([o0, o1, o2])
    # 列方向ベクトル
    col_direction = np.array([o3, o4, o5])

    # 各マスクに対して3次元座標を計算
    for i in range(5):
        # ピクセル座標を取得
        mean_y = middle_s_all_df.loc[j, f'mean_bfr_cut_y_{i}']  # 行方向（R）
        mean_x = middle_s_all_df.loc[j, f'mean_bfr_cut_x_{i}']  # 列方向（C）

        # 3次元座標を計算
        patient_pos = np.array([x_pos, y_pos, z_pos]) + \
                      (pixel_sp_y * mean_x) * row_direction + \
                      (pixel_sp_z * mean_y) * col_direction

        # 計算した3次元座標をカラムに格納
        middle_s_all_df.loc[j, f'xx_{i}'] = patient_pos[0]
        middle_s_all_df.loc[j, f'yy_{i}'] = patient_pos[1]
        middle_s_all_df.loc[j, f'zz_{i}'] = patient_pos[2]


In [ ]:
# DataFrameを保存するパス
save_path = '/content/drive/MyDrive/RSNA_csv/middle_s_all_df_rev.pkl'

# DataFrameをpkl形式で保存
middle_s_all_df.to_pickle(save_path)

print(f"DataFrame has been saved to {save_path}")

# 病気を判定するモデル学習部分

In [ ]:
import pickle
with open("/content/drive/MyDrive/RSNA_csv/grouped_df_rev.pkl", 'rb') as f:
  grouped_df=pickle.load(f)

In [ ]:
unique_studies = grouped_df['study_id'].unique()
study_mapping = {study: (i % 5) + 1 for i, study in enumerate(unique_studies)}

# study_idごとにfoldをわける
grouped_df['series_description2'] = grouped_df['study_id'].map(study_mapping)


In [ ]:
tdf2=grouped_df[grouped_df['series_description2'] != fold]
vdf2=grouped_df[grouped_df['series_description2'] == fold]

In [ ]:
vdf2.to_pickle("/content/drive/MyDrive/RSNA_csv/grouped_df_sagt1_vdf2.pkl")

In [ ]:
import albumentations as A #データ拡張の処理をする

AUG_PROB = 0.75 #75%の確率でデータ拡張する
transforms_train = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), p=AUG_PROB),#明るさやコントラストを20%の確率で変更
    A.OneOf([
        A.MotionBlur(blur_limit=5),#動きによるぼかしをいれる
        # A.MedianBlur(blur_limit=5),
        A.GaussianBlur(blur_limit=5),#ガウスフィルタによるぼかしを入れる
        A.GaussNoise(var_limit=(5.0, 30.0)),#ランダムなガウスノイズを入れる
    ], p=AUG_PROB),

    A.OneOf([
        A.OpticalDistortion(distort_limit=1.0),#光学的な歪みを実現
        A.GridDistortion(num_steps=5, distort_limit=1.),#グリッドごとにゆがませる
        A.ElasticTransform(alpha=3),#弾性変形を適用
    ], p=AUG_PROB),

    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=AUG_PROB),#平行移動、拡大縮小、回転
  #  A.CoarseDropout(max_holes=15, max_height=30, max_width=30, min_holes=1, min_height=8, min_width=8, p=AUG_PROB),
])

In [ ]:
patch_size = 90 #切り取るサイズ

In [ ]:
import torch.nn.functional as F
class ViT_T1_Dataset(Dataset):
    def __init__(self, df, UNet,transform=None, VALID=False, P=patch_size, alpha=0):
        self.data = df
        self.UNet = UNet
        self.VALID = VALID
        self.P = P
        self.alpha = alpha
        self.transform = transform

    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
      x = np.zeros((8, self.P, self.P), dtype=np.float32)#まずは全部0が入っているものを作るサイズはpatch_size*patch_size
      non_zero_slice=[]

      for i in range(8):

        sample = '/content/train_images/'
        sample = sample+str(self.data.iloc[index]['study_id'])+'/'+str(self.data.iloc[index]['series_id'])+'/'+str(self.data.iloc[index][f'instance_number_{i}'])+'.dcm'

        image = pydicom.dcmread(sample).pixel_array
        H,W = image.shape
        # centers = torch.as_tensor([x for x in row[coor]]).view(5,2).float()
        # 正方形に切り取る
        if H > W:
            d = W
            h = (H - d)//2
            image = image[h:h+d]
            # centers[:,1] -= h
            H = W
        elif H < W:
            d = H
            w = (W - d)//2
            image = image[:,w:w+d]
            # centers[:,0] -= w
            W = H
        image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
        image = torch.as_tensor(image/(np.max(image))).unsqueeze(0).unsqueeze(0).float().to(device)

        # OUT = 0
        # with torch.no_grad():
                # for rot in [0,1,2,3]:
                        # OUT += torch.rot90(self.UNet(torch.rot90(image,rot,dims=[-2, -1])),-rot,dims=[-2, -1])
        OUT=self.UNet(image)
        OUT = (OUT > TH)[0]#閾値0.5でマスクを0と1に分ける
        c = (OUT.unsqueeze(1)*idx_map[0]).view(5,2,PATCH_SIZE*PATCH_SIZE).sum(-1)
        d = OUT.view(5,PATCH_SIZE*PATCH_SIZE).sum(-1)
        m = d > 0
        c[m] = (c[m]/(d[m]).unsqueeze(-1)).long()
        c[~m] = self.P

        image_slices = []

        for xy in c:
          y_start = max(0, xy[1] - self.P // 2)
          y_end = min(512,xy[1] + self.P - self.P // 2)
          x_start = max(0, xy[0] - self.P // 2)
          x_end = min(512,xy[0] + self.P - self.P // 2)
          if (y_end - y_start == self.P) and (x_end - x_start == self.P):#切り取りの部分が途中で切れていないない場合
            slice_img = image[0, 0, y_start:y_end, x_start:x_end]
            non_zero_slice.append(slice_img)
            image_slices.append(slice_img)
          else:#切り取りの部分が途中で切れている場合
            zero_slice = torch.zeros((self.P, self.P), device=image.device)
            image_slices.append(zero_slice)
            # print(f"Skipped slice due to incorrect size: {(y_end - y_start, x_end - x_start)}")
            # print(f"Slice coordinates: {(xy[1], xy[0])}")

# スライスがあればスタックする
        if image_slices:
          try:
            image = torch.stack(image_slices)
          except RuntimeError as e:
            print(f"Error: {e}")
        else:
          print("No valid slices available for stacking.")

        # if not self.VALID: image = augment_image(image,self.alpha)
        x[i,...]=image[self.data.iloc[index]['level']].cpu().numpy()
      for i in range(8):
        if (x[i,...].sum() == 0) and (len(non_zero_slice)>0):
          x[i,...] = non_zero_slice[0].cpu().numpy()
        else:
          pass
        #nothing
      if self.transform is not None:
        x = self.transform(image=x)['image']#albumentationを適用
      x=torch.as_tensor(x).float()
      x = F.interpolate(x.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)#モデルに適用するために224*224のサイズにする
      label = torch.as_tensor(labels[self.data.iloc[index]['label']])

      return [x.to(device),label.to(device)]


In [ ]:
def myLoss(preds,target):
    return nn.CrossEntropyLoss(weight=torch.as_tensor([1.,2.,4.]).to(device))(preds+1e-12,target)#コンペの評価指標が重み[1.,2.,4.]だったためこのように設定

In [ ]:
import timm
class ViT(nn.Module):
    def __init__(self, num_classes):
        super(ViT, self).__init__()
        # ここにViTのモデルアーキテクチャを実装
        self.vit = timm.create_model('eva02_base_patch14_224', pretrained=True, num_classes=num_classes,in_chans=8,features_only=False,global_pool='avg')#timmからモデルを持ってくる
        # self.model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # self.vit.head.drop.p = 0.5
        #edgenext_base.in21k_ft_in1k
        # self.vit.features.conv0=nn.Conv2d(5, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        # self.new_model=nn.Sequential(*self.layer)
    def forward(self, x):
        # ここに順伝播の処理を実装
        return self.vit(x)

In [ ]:
from fastai.callback.core import Callback

class SaveModelCallback(Callback):#モデルをエポックごとに保存するためのコールバック関数
    def __init__(self, every_epoch=False, path='models', fname='model',with_opt=False):
        self.every_epoch = every_epoch
        self.path = path
        self.fname = fname
        self.with_opt = with_opt

    def after_epoch(self):
        # エポックごとにモデルを保存する
        if self.every_epoch:
            self.learn.save(f'{self.path}/{self.fname}_ep_{self.epoch}')

save_model_cb = SaveModelCallback(with_opt=True,every_epoch=True, path='/content/drive/MyDrive/RSNA_csv', fname=f"eva_p_90_ch_8_f_{fold}")

In [ ]:
INF['EPOCHS']=40

In [ ]:
from torch.optim import AdamW
if 1:
    seed_everything(SEED)
    with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+"1"+".pkl", 'rb') as f:
      UNet=pickle.load(f)
    tds = ViT_T1_Dataset(tdf2,UNet,transform=transforms_train)#データセット関数からデータ拡張を適用して読み込む
    vds = ViT_T1_Dataset(vdf2,UNet,VALID=True)#データセット関数から読み込む
    tdl = torch.utils.data.DataLoader(tds, batch_size=16, shuffle=True, drop_last=True)#データローダーの作成
    vdl = torch.utils.data.DataLoader(vds, batch_size=16, shuffle=False)#データローダーの作成

    dls = DataLoaders(tdl,vdl)#fastai用のデータローダーの作成

    n_iter = len(tds)//INF['BS']

    model = ViT(num_classes=3)#モデル作成
    model.to(device)
    learn = Learner(#Learnerを作成
        dls,
        model,
        lr=INF['LR'],
        loss_func=myLoss,
        cbs=[
            save_model_cb,
            GradientClip,
            ShowGraphCallback(),#学習曲線を表示
            # alpha_cb
        ]
    )
    learn.fit_one_cycle(INF['EPOCHS'],wd=1e-2)
    with open('/content/drive/MyDrive/RSNA_csv/'+"VIT_"+str(fold)+".pkl", 'wb') as f:
      pickle.dump(model, f)

In [ ]:
from fastai.vision.all import *
from torch.optim import AdamW
import pickle

if 1:
    seed_everything(SEED)

    # モデルの読み込み
    with open('/content/drive/MyDrive/RSNA_csv/'+"SEG_"+"1"+".pkl", 'rb') as f:
        UNet = pickle.load(f)

    # Dataset 作成
    tds = ViT_T1_Dataset(tdf2, UNet, transform=transforms_train)
    vds = ViT_T1_Dataset(vdf2, UNet, VALID=True)
    tdl = torch.utils.data.DataLoader(tds, batch_size=16, shuffle=True, drop_last=True)
    vdl = torch.utils.data.DataLoader(vds, batch_size=16, shuffle=False)

    # DataLoaders 設定
    dls = DataLoaders(tdl, vdl)
    n_iter = len(tds)//INF['BS']

    # モデルと学習設定
    model = ViT(num_classes=3)
    model.to(device)

    # Learner 作成
    learn = Learner(
        dls,
        model,
        lr=INF['LR'],
        loss_func=myLoss,
        cbs=[
            save_model_cb,
            GradientClip,
            # ShowGraphCallback(),
            # alpha_cb
        ],# オプティマイザをAdamWに設定
    )

    # モデルを読み込む (with_opt=True でオプティマイザの状態も復元)
    learn.load('/content/drive/MyDrive/RSNA_csv/eva_p_90_ch_8_f_1_ep_12', with_opt=True,device=device)
    # Fine-tune モードで追加学習
    learn.fit_one_cycle(40, start_epoch=13,wd=1e-2)


#混同行列の算出

In [ ]:
vdf2=pd.read_pickle("/content/drive/MyDrive/RSNA_csv/grouped_df_sagt1_vdf2.pkl")

In [ ]:
vdf2[['description','study_id','label','level']]

In [ ]:
ALL_CONDITIONS = sorted(["left_neural_foraminal_narrowing", "right_neural_foraminal_narrowing"])
LEVELS = ["l1_l2", "l2_l3", "l3_l4", "l4_l5", "l5_s1"]


In [ ]:
# Pre-populate results df
import glob
import os
study_ids = vdf2['study_id'].unique().tolist()

results_df = pd.DataFrame({"row_id":[], "normal_mild": [], "moderate": [], "severe": []})
for study_id in study_ids:
    for condition in ALL_CONDITIONS:
        for level in LEVELS:
            row_id = f"{study_id}_{condition}_{level}"
            results_df = results_df._append({"row_id": row_id, "normal_mild": 1/3, "moderate": 1/3, "severe": 1/3}, ignore_index=True)

In [ ]:
results_df

In [ ]:
grouped_df=vdf2
with open("/content/drive/MyDrive/RSNA_csv/SEG_1.pkl", 'rb') as f:
  model1=pickle.load(f)

In [ ]:
import torch.nn.functional as F
class ViT_T1_Dataset(Dataset):
    def __init__(self, df, UNet, VALID=False, P=patch_size, alpha=0):
        self.data = df
        self.UNet = UNet
        self.VALID = VALID
        self.P = P
        self.alpha = alpha

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = np.zeros((8, self.P, self.P), dtype=np.float32)
        non_zero_slice=[]

        for i in range(8):

          sample = '/content/train_images/'
          sample = sample+str(self.data.iloc[index]['study_id'])+'/'+str(self.data.iloc[index]['series_id'])+'/'+str(self.data.iloc[index][f'instance_number_{i}'])+'.dcm'

          image = pydicom.dcmread(sample).pixel_array
          H,W = image.shape
          # centers = torch.as_tensor([x for x in row[coor]]).view(5,2).float()
          # By plane resizing I've been distorting the proportions
          if H > W:
              d = W
              h = (H - d)//2
              image = image[h:h+d]
              # centers[:,1] -= h
              H = W
          elif H < W:
              d = H
              w = (W - d)//2
              image = image[:,w:w+d]
              # centers[:,0] -= w
              W = H
          image = cv2.resize(image,(PATCH_SIZE,PATCH_SIZE))
          image = torch.as_tensor(image/(np.max(image))).unsqueeze(0).unsqueeze(0).float().to(device)

          # OUT = 0
          # with torch.no_grad():
                  # for rot in [0,1,2,3]:
                          # OUT += torch.rot90(self.UNet(torch.rot90(image,rot,dims=[-2, -1])),-rot,dims=[-2, -1])
          OUT=self.UNet(image)
          OUT = (OUT > TH)[0]
          c = (OUT.unsqueeze(1)*idx_map[0]).view(5,2,PATCH_SIZE*PATCH_SIZE).sum(-1)
          d = OUT.view(5,PATCH_SIZE*PATCH_SIZE).sum(-1)
          m = d > 0
          c[m] = (c[m]/(d[m]).unsqueeze(-1)).long()
          c[~m] = self.P # I have to find a better solution

          image_slices = []

          for xy in c:
            y_start = max(0, xy[1] - self.P // 2)
            y_end = min(512,xy[1] + self.P - self.P // 2)
            x_start = max(0, xy[0] - self.P // 2)
            x_end = min(512,xy[0] + self.P - self.P // 2)

    # スライスが有効なサイズを持つか確認
            if (y_end - y_start == self.P) and (x_end - x_start == self.P):
              slice_img = image[0, 0, y_start:y_end, x_start:x_end]
              non_zero_slice.append(slice_img)
              image_slices.append(slice_img)
            else:
              zero_slice = torch.zeros((self.P, self.P), device=image.device)
              image_slices.append(zero_slice)
              # print(f"Skipped slice due to incorrect size: {(y_end - y_start, x_end - x_start)}")
              # print(f"Slice coordinates: {(xy[1], xy[0])}")

# スライスがあればスタックする
          if image_slices:
            try:
              image = torch.stack(image_slices)
            except RuntimeError as e:
              print(f"Error: {e}")
          else:
            print("No valid slices available for stacking.")

          # if not self.VALID: image = augment_image(image,self.alpha)
          x[i,...]=image[self.data.iloc[index]['level']].cpu().numpy()

        for i in range(8):
          if (x[i,...].sum() == 0) and (len(non_zero_slice)>0):
            x[i,...] = non_zero_slice[0].cpu().numpy()
          else:
            pass
          #nothing
        x=torch.as_tensor(x).float()
        x = F.interpolate(x.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
        # label = torch.as_tensor(labels[self.data.iloc[index]['label']])

        return [x.to(device),m.to(device)]


In [ ]:
model = ViT(num_classes=3)

model = model.to(device)

checkpoint = torch.load('/content/drive/MyDrive/RSNA_csv/eva_p_90_ch_8_f_1_ep_17.pth')
model.load_state_dict(checkpoint['model'])

model.eval()

for i in range(8):  # Iterating from 0 to 4
    column_name = f'instance_number_{i}'
    grouped_df[column_name].fillna(1, inplace=True)  # Replacing NaN with 1


test_s=ViT_T1_Dataset(grouped_df,model1)


In [ ]:
from tqdm import tqdm
grouped_df = grouped_df.reset_index(drop=True)

study_id=grouped_df['study_id'].iloc[0]

with torch.no_grad():
    for i in tqdm(range(len(test_s)), desc="Processing predictions"):
      try:
        row = grouped_df.iloc[i]
        x=test_s.__getitem__(i)[0].unsqueeze(0)
        probabilities = F.softmax(model(x), dim=1)
      #   print(probabilities)
        if row['description']=='Left Neural Foraminal Narrowing':
          if row['level']==0:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l1_l2",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l1_l2",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l1_l2",'severe']=probabilities[0][2].item()
          elif row['level']==1:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l2_l3",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l2_l3",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l2_l3",'severe']=probabilities[0][2].item()
          elif row['level']==2:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l3_l4",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l3_l4",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l3_l4",'severe']=probabilities[0][2].item()
          elif row['level']==3:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l4_l5",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l4_l5",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l4_l5",'severe']=probabilities[0][2].item()
          elif row['level']==4:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l5_s1",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l5_s1",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_left_neural_foraminal_narrowing_l5_s1",'severe']=probabilities[0][2].item()
        elif row['description']=='Right Neural Foraminal Narrowing':
          if row['level']==0:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l1_l2",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l1_l2",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l1_l2",'severe']=probabilities[0][2].item()
          elif row['level']==1:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l2_l3",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l2_l3",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l2_l3",'severe']=probabilities[0][2].item()
          elif row['level']==2:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l3_l4",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l3_l4",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l3_l4",'severe']=probabilities[0][2].item()
          elif row['level']==3:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l4_l5",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l4_l5",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l4_l5",'severe']=probabilities[0][2].item()
          elif row['level']==4:
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l5_s1",'normal_mild']=probabilities[0][0].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l5_s1",'moderate']=probabilities[0][1].item()
            results_df.loc[results_df['row_id']==f"{row['study_id']}_right_neural_foraminal_narrowing_l5_s1",'severe']=probabilities[0][2].item()
      except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
vdf2['label'].value_counts()

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report

# マッピングのための辞書を用意
description_to_condition = {
    "Left Neural Foraminal Narrowing": "left_neural_foraminal_narrowing",
    "Right Neural Foraminal Narrowing": "right_neural_foraminal_narrowing",
}

level_to_level_t = {
    0: "l1_l2",
    1: "l2_l3",
    2: "l3_l4",
    3: "l4_l5",
    4: "l5_s1",
}

true_label_to_true_label_t={
    "Normal/Mild":"normal_mild",
    "Moderate":"moderate",
    "Severe":"severe"
}

# vdf2 の正解データを results_df にマッピング
true_labels = []
predicted_labels = []

for _, row in vdf2.iterrows():
    study_id = row['study_id']
    description = row['description']
    level = row['level']
    level_t=level_to_level_t[level]
    condition = description_to_condition[description]


    # 正解ラベル
    true_label = row['label']  # 正解ラベルがここにあると仮定
    true_label_t=true_label_to_true_label_t[true_label]
    true_labels.append(true_label_t)

    # 予測確率に基づいて予測ラベルを取得
    row_id = f"{study_id}_{condition}_{level_t}"
    predicted_row = results_df[results_df['row_id'] == row_id]

    if not predicted_row.empty:
        # 最大確率の列名が予測ラベル
        predicted_label = predicted_row[['normal_mild', 'moderate', 'severe']].idxmax(axis=1).values[0]
        predicted_labels.append(predicted_label)
    else:
        print(f"Warning: No prediction found for row_id {row_id}")
        predicted_labels.append(None)  # 空の場合は None を設定

# None を除外
true_labels_filtered = [t for t, p in zip(true_labels, predicted_labels) if p is not None]
predicted_labels_filtered = [p for p in predicted_labels if p is not None]

# 混同行列を計算
conf_matrix = confusion_matrix(true_labels_filtered, predicted_labels_filtered, labels=["normal_mild", "moderate", "severe"])

# 結果表示
print("Confusion Matrix:")
print(conf_matrix)

# 詳細レポート
print("\nClassification Report:")
print(classification_report(true_labels_filtered, predicted_labels_filtered, target_names=["normal_mild", "moderate", "severe"]))
